In [1]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

# Append the path to Pydoop to sys.path
#sys.path.append("/usr/local/lib/python3.8/dist-packages")

Spark Configuration

In [2]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
# spark_session = SparkSession.builder\
#     .master("local[2]")\
#     .appName("pseudo_spark_nora")\
#     .getOrCreate()

# Set Hadoop configuration directory
#os.environ['HADOOP_CONF_DIR'] = '/path/to/hadoop/conf'
#spark_session.stop()
#os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

spark_session = SparkSession.builder \
    .appName("HDFS_Connection_Test_nora") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# num executors 
# executor memory RAM 


# spark_session = SparkSession.builder \
#     .appName("HDFS_Connection_Test_nora") \
#     .master("yarn") \
#     .config("spark.executor.instances", "2") \
#     .config("spark.hadoop.fs.default.name", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.defaultFS", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
#     .config("spark.hadoop.dfs.server.namenode.class", "org.apache.hadoop.hdfs.server.namenode.NameNode") \
#     .config("spark.hadoop.conf", "org.apache.hadoop.hdfs.HdfsConfiguration") \
#     .getOrCreate()

#.config("spark.dynamicAllocation.enabled", False)\
#.config("spark.cores.max", 4)\


#.config("spark.jars.packages", "LLNL:spark-hdf5:0.0.4") \
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/15 03:00:05 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
/home/hadoop/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [36]:
#path = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI/TRAAAAW128F429D538.asci'
#path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI_analysis_songs/'

# path_0 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_0/'
# path_1 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_1/'
# path_2 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_2/'
# path_3 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_3/'
# path_4 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_4/'
# path_5 = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI_SUB/subfolder_5/'
# paths = [path_0, path_1, path_2, path_3, path_4, path_5]

path = 'hdfs://master-node:9000/user/hadoop/MSD_ASCI/'

In [37]:
file_contents = spark_context.wholeTextFiles(path).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))
#file_contents = spark_context.wholeTextFiles(','.join(paths)).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))

24/03/15 03:08:59 INFO MemoryStore: Block broadcast_17 stored as values in memory (estimated size 344.0 KiB, free 364.6 MiB)
24/03/15 03:08:59 INFO MemoryStore: Block broadcast_17_piece0 stored as bytes in memory (estimated size 33.3 KiB, free 364.6 MiB)
24/03/15 03:08:59 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on master-node:46797 (size: 33.3 KiB, free: 366.1 MiB)
24/03/15 03:08:59 INFO SparkContext: Created broadcast 17 from wholeTextFiles at NativeMethodAccessorImpl.java:0


In [38]:
split_file_contents = file_contents.map(lambda x: x[0].split(','))

In [39]:
selected_elements = split_file_contents.map(lambda x: [float(x[i]) for i in [3, 4, 26, 23, 24, 25, 27, 28, 29]])

In [40]:
#selected_elements.first()

## Data Frame

In [41]:
schema = StructType([
    StructField("duration", FloatType(), True),
    StructField("end_of_fade_in", FloatType(), True),
    StructField("start_of_fade_out", FloatType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", FloatType(), True),
    StructField("mode_confidence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("time_signature_confidence", FloatType(), True)
])

In [42]:
df = spark_session.createDataFrame(selected_elements, schema)

In [43]:
num_rows = df.count()
print("Number of rows in the DataFrame:", num_rows)

24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_16_piece0 on master-node:46797 in memory (size: 11.1 KiB, free: 366.1 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_16_piece0 on worker-node-2:41937 in memory (size: 11.1 KiB, free: 366.2 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_13_piece0 on master-node:46797 in memory (size: 6.0 KiB, free: 366.1 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_13_piece0 on worker-node-2:41937 in memory (size: 6.0 KiB, free: 366.2 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_12_piece0 on worker-node-2:41937 in memory (size: 12.8 KiB, free: 366.2 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_12_piece0 on worker-node-1:45621 in memory (size: 12.8 KiB, free: 366.2 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadcast_12_piece0 on master-node:46797 in memory (size: 12.8 KiB, free: 366.1 MiB)
24/03/15 03:09:07 INFO BlockManagerInfo: Removed broadca

Number of rows in the DataFrame: 10000


24/03/15 03:12:17 INFO TaskSetManager: Finished task 1.0 in stage 19.0 (TID 20) in 188572 ms on worker-node-1 (executor 2) (2/2)
24/03/15 03:12:17 INFO YarnScheduler: Removed TaskSet 19.0, whose tasks have all completed, from pool 
24/03/15 03:12:17 INFO DAGScheduler: ShuffleMapStage 19 (count at NativeMethodAccessorImpl.java:0) finished in 188.591 s
24/03/15 03:12:17 INFO DAGScheduler: looking for newly runnable stages
24/03/15 03:12:17 INFO DAGScheduler: running: Set()
24/03/15 03:12:17 INFO DAGScheduler: waiting: Set()
24/03/15 03:12:17 INFO DAGScheduler: failed: Set()
24/03/15 03:12:17 INFO SparkContext: Starting job: count at NativeMethodAccessorImpl.java:0
24/03/15 03:12:17 INFO DAGScheduler: Got job 15 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/15 03:12:17 INFO DAGScheduler: Final stage: ResultStage 21 (count at NativeMethodAccessorImpl.java:0)
24/03/15 03:12:17 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 20)
24/03/15 03:12:17 I

In [44]:
df.show()

24/03/15 03:12:17 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/15 03:12:17 INFO DAGScheduler: Got job 16 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/15 03:12:17 INFO DAGScheduler: Final stage: ResultStage 22 (showString at NativeMethodAccessorImpl.java:0)
24/03/15 03:12:17 INFO DAGScheduler: Parents of final stage: List()
24/03/15 03:12:17 INFO DAGScheduler: Missing parents: List()
24/03/15 03:12:17 INFO DAGScheduler: Submitting ResultStage 22 (MapPartitionsRDD[63] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/15 03:12:17 INFO MemoryStore: Block broadcast_20 stored as values in memory (estimated size 28.4 KiB, free 364.7 MiB)
24/03/15 03:12:17 INFO MemoryStore: Block broadcast_20_piece0 stored as bytes in memory (estimated size 11.6 KiB, free 364.7 MiB)
24/03/15 03:12:17 INFO BlockManagerInfo: Added broadcast_20_piece0 in memory on master-node:46797 (size: 11.6 KiB, free: 

+--------+--------------+-----------------+--------+----+---------------+-------+--------------+-------------------------+
|duration|end_of_fade_in|start_of_fade_out|loudness|mode|mode_confidence|  tempo|time_signature|time_signature_confidence|
+--------+--------------+-----------------+--------+----+---------------+-------+--------------+-------------------------+
| 218.932|         0.247|          218.932| -11.197| 0.0|          0.636| 92.198|           4.0|                    0.778|
| 148.035|         0.148|          137.915|  -9.843| 0.0|           0.43|121.274|           4.0|                    0.384|
| 177.475|         0.282|          172.304|  -9.689| 1.0|          0.565| 100.07|           1.0|                      0.0|
| 233.404|           0.0|          217.124|  -9.013| 1.0|          0.749|119.293|           4.0|                      0.0|
| 209.606|         0.066|          198.699|  -4.501| 1.0|          0.371|129.738|           4.0|                    0.562|
| 267.702|      

24/03/15 03:12:18 INFO TaskSetManager: Finished task 0.0 in stage 22.0 (TID 22) in 1118 ms on worker-node-2 (executor 1) (1/1)
24/03/15 03:12:18 INFO YarnScheduler: Removed TaskSet 22.0, whose tasks have all completed, from pool 
24/03/15 03:12:18 INFO DAGScheduler: ResultStage 22 (showString at NativeMethodAccessorImpl.java:0) finished in 1.133 s
24/03/15 03:12:18 INFO DAGScheduler: Job 16 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/15 03:12:18 INFO YarnScheduler: Killing all running tasks in stage 22: Stage finished
24/03/15 03:12:18 INFO DAGScheduler: Job 16 finished: showString at NativeMethodAccessorImpl.java:0, took 1.139434 s


# Calculation (Average)

In [45]:
average_values = df.agg({'duration': 'avg', 'end_of_fade_in': 'avg', 'start_of_fade_out': 'avg', 
                         'loudness': 'avg', 'mode': 'avg', 'mode_confidence': 'avg', 
                         'tempo': 'avg', 'time_signature': 'avg', 
                         'time_signature_confidence': 'avg'}).collect()[0]

24/03/15 03:12:18 INFO DAGScheduler: Registering RDD 65 (collect at /tmp/ipykernel_300399/2441391651.py:1) as input to shuffle 6
24/03/15 03:12:18 INFO DAGScheduler: Got map stage job 17 (collect at /tmp/ipykernel_300399/2441391651.py:1) with 2 output partitions
24/03/15 03:12:18 INFO DAGScheduler: Final stage: ShuffleMapStage 23 (collect at /tmp/ipykernel_300399/2441391651.py:1)
24/03/15 03:12:18 INFO DAGScheduler: Parents of final stage: List()
24/03/15 03:12:18 INFO DAGScheduler: Missing parents: List()
24/03/15 03:12:18 INFO DAGScheduler: Submitting ShuffleMapStage 23 (MapPartitionsRDD[65] at collect at /tmp/ipykernel_300399/2441391651.py:1), which has no missing parents
24/03/15 03:12:18 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 50.9 KiB, free 364.7 MiB)
24/03/15 03:12:18 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 19.1 KiB, free 364.6 MiB)
24/03/15 03:12:18 INFO BlockManagerInfo: Added broadcast_21_pi

In [46]:
avg_df = pd.DataFrame([average_values.asDict()])

In [47]:
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 10000


In [48]:
avg_df

,avg(duration),avg(tempo),avg(end_of_fade_in),avg(start_of_fade_out),avg(time_signature),avg(time_signature_confidence),avg(mode_confidence),avg(mode),avg(loudness)
0,238.507519,122.915449,0.758616,229.975465,3.5648,0.509937,0.477784,0.6911,-10.485669


# STOP SPARK

In [ ]:
# release the cores
spark_context.stop()